## Imports:

In [3]:
import os
import re
import numpy as np
from time import time
import pybullet as p
import copy

from Environments.environment_sim2 import Environment
from Config.constants import (WORKSPACE_LIMITS)
from V1_destination_prediction.Test_cases.tc3_no_bottom import TestCase1

#Importing Python Libraries
import glob
import imp
import math
import gc
from sre_constants import SUCCESS
import datetime
import pybullet as p
import cv2
import numpy as np
from graphviz import Digraph
import argparse
import random
import matplotlib.pyplot as plt
from time import sleep
import copy
import json
from collections import namedtuple, deque


#Importing Torch libraries
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from Environments.environment_sim2 import Environment
import Environments.utils as env_utils
from Environments.utils import sample_goal, get_pose_distance

# --------------------- Imports from V1_destination_prediction: -------------------------- #

from V1_destination_prediction.Test_cases.tc3_no_bottom_sim import TestCase1
#from V1_destination_prediction.Test_cases.tc3_no_bottom import TestCase1

# ---------------------------------------------------------------------------------------- #

#Imports from V2_next_best_action:
from V2_next_best_action.models.dqn_v2 import pushDQN2

#Imports from create_env.py:
from create_env import get_push_start, get_max_extent_of_target_from_bottom, get_push_start_POSMAX


C:\Users\kallo\AppData\Local\Temp\ipykernel_24096\1770363567.py:14: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
#Initializing a pyTorch device, on which torch tensors will be allocated
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Is GPU Available? ", torch.cuda.is_available())

Is GPU Available?  True


In [3]:
ITER = 0   

#Define the state space:
n_observations = 6 # 3 for initial state, 3 for goal state

#Define the action space:
n_actions = 16 # 16 push + 1 grasp ???How are the 16 push actions distributed, I mean how are they applied and where?

#Initialize the DQN policy (states --> action values) neural network, set it to use cuda, and assign the cuda device to be used.
policy_net = pushDQN2(n_observations, n_actions, use_cuda=True).to(device)

#???Note: Do not confuse the "policy_net" for being the policy, its actually the Q-function I guess, it gives us the value of
#each state-action pair.

"""PYTORCH MODEL LOADING PROCESS:"""

#load a previous checkpoint .pt file
checkpoint = torch.load('V2_next_best_action/models/model_checkpoints/dqnv2/model7/5350.pt')
#Assign the parameters from the checkpoint to the policy network
policy_net.load_state_dict(checkpoint)

print("Policy Model Definition: ")
#Evaluate and print model
policy_net.eval()

Policy Model Definition: 


pushDQN2(
  (layer1): Linear(in_features=6, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=16, bias=True)
)

In [4]:
def select_action(state):
    '''Select the next best action 
    state: tensor(shape=(6))
    '''
    with torch.no_grad():  #Make sure that gradients are not calculated for parameters, because they aren't needed here
        
        #Pass the state through the state-action value network, then find the maximum value, and index into the 1st element to
        #get the action index of the maximum value. Then reshape the tensor to (1,1).
        return policy_net(state).max(1)[1].view(1, 1)

    
def get_reward2(prev_state, current_state):
    '''
    prev_state: (x1, y1, theta1, x2, y2, theta2)
    current_state: (x3, y3, theta3, _, _, _)
    '''

    #???
    
    #Difference between norms of previous state's error in between goal coordinates and initial coordinates, and the same error 
    #for current state
    pos_diff = np.linalg.norm(prev_state[0:2] - prev_state[3:5]) - np.linalg.norm(current_state[0:2] - prev_state[3:5])
    
    #Difference between norms of previous state's initial theta and goal theta, and the same error for current state:
    orn_diff = np.linalg.norm(prev_state[2:3] - prev_state[5:6]) - np.linalg.norm(current_state[2:3] - prev_state[5:6])
    
    #Reward is the weighted sum of difference in position and difference in orientation, with weights = 1, 0.1
    reward = pos_diff + 0.1*orn_diff  
    
    return reward

In [5]:
def get_joint_cost(all_joints):
    '''
    all_joints = np.array(n, 6)
    '''
    print(len(all_joints), len(all_joints[0]))
    cost = 0
    for i in range(1, len(all_joints)-1):
        temp_cost = 0
        for j in range(6):
            temp_cost += ((all_joints[i][j] - all_joints[i-1][j]))**2
        cost += np.sqrt(temp_cost)

    return cost

### Get best trajectory:

In [6]:
min_cost = 1e+500

save_folder = "Run1"

for _, _, file_names in os.walk("Saves/" + save_folder):
    for file_name in file_names:
        if re.search('costs', file_name):
            cost = np.load("Saves/" + save_folder + "/" + file_name)
            for i in range(cost.size):
                if cost[i]< min_cost:
                    min_cost = cost[i]
                    index = i
                    time_stamp = re.findall('s.*n', file_name)[0][3:-1]

best_cost = np.load("Saves/" + save_folder + "/costs" + time_stamp + 'npy')[index]
best_traj = np.load("Saves/" + save_folder + "/latest_trajectory_list" + time_stamp + 'npy')[index]
print(best_traj)
                

[[ 0.35       -0.15      ]
 [ 0.39985316 -0.19574957]
 [ 0.52857485 -0.28002247]
 [ 0.68985863 -0.32289595]
 [ 0.77719896 -0.24261104]
 [ 0.72106202 -0.018988  ]
 [ 0.66        0.12      ]]


In [2]:
min_cost = 1e+500

save_folder = "Run17_3cylinders"

for _, _, file_names in os.walk("Results/" + save_folder + "/costs_and_trajectories/"):
    for file_name in file_names:
        if re.search('costs', file_name):
            cost = np.load("Results/" + save_folder + "/costs_and_trajectories/" + file_name)
            for i in range(cost.size):
                if cost[i]< min_cost:
                    min_cost = cost[i]
                    index = i
                    time_stamp = re.findall('s.*n', file_name)[0][3:-1]

best_cost = np.load("Results/" + save_folder + "/costs_and_trajectories" + "/costs" + time_stamp + 'npy')[index]
best_traj = np.load("Results/" + save_folder + "/costs_and_trajectories" +  "/latest_trajectory_list" + time_stamp + 'npy')[index]
print(best_traj)
                

NameError: name 'os' is not defined

### Simulate:

In [8]:
def run_policy(env, cur_traj, START_PT, threshold_d, WORKSPACE_LIMITS, MAX_PUSH_ITER=30, obstacle_num = 1):
    '''
    '''
    #Angular directions along which the object can be pushed (16 standard directions), divided equally from 360 degrees (2pi)
    push_directions = [0, np.pi/8, np.pi/4, 3*np.pi/8,
                    np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8, 
                    np.pi, 9*np.pi/8, 5*np.pi/4, 11*np.pi/8,  
                    3*np.pi/2, 13*np.pi/8, 7*np.pi/4, 15*np.pi/8] # 16 standard directions
    
    #Reset the environment:
    #This resets objects, sets gravity, resets simulation timer, loads the plane, loads workspace table, sets the Physical
    #Dynamics like friction, damping etc., loads in the URDF and stores it joint ids in the environment
    env.reset()
    
    #Create a Test Case:
    testcase1 = TestCase1(env)
    
    #Sets up the task on the workspace, with a target object and a bottom object to be picked up by the robot. 
    #body_ids has the object ID of the target object, and sucess is true if the target is within bounds of the 
    
    # ----------------------------------------------------------- #
    
    body_ids, sucess = testcase1.create_specific_test_case(targetPos=START_PT, obstacle_config = "Results/" + save_folder + "/obstacle_config.npy")
    #body_ids, sucess = testcase1.create_specific_test_case(targetPos=START_PT, scene_id = 0)
    
    # ----------------------------------------------------------- #
    
    #body_ids[0] is the target object handle, 
    targetPos, targetOrn = p.getBasePositionAndOrientation(body_ids[0])
    
    #Initialize empty lists for marker poses and marker yaws:
    marker_poses = []
    marker_yaws = []
    
    INVALID_POS = False
    
    for k in range(len(cur_traj)-1):
        
        #copy the position of target object to position of marker
        marker_pos = copy.deepcopy(np.array(targetPos))
        
        #Assign the (x,y) of the marker position to the (x,y) of the (k+1)th point of the trajectory
        #Remember that (k+1) goes from 1 to TRAJ_LEN, that is, from the second point to last point of trajectory.
        marker_pos[0] = cur_traj[k+1][0]                      
        marker_pos[1] = cur_traj[k+1][1]
        
        #Append the marker position to the list of marker positions
        marker_poses.append(marker_pos)
        
        #Sample the yaw angle of the marker position from a uniform distribution and append it to the list of marker yaw angles
        marker_yaw = np.random.uniform(low=0, high=np.pi)
        marker_yaws.append(marker_yaw)

    ###################################################################################################
    #??? This section should not even run since INVALID_POS will always be False.
    if INVALID_POS == True:
        cost = OUT_OF_BOUNDS_COST
        # sample_costs.append(cost)
        all_joints = []
        # sample_joint_trajectories.append(all_joints)
        print(f"{ITER}. Cost: {cost} -------------------------")  
        return cost, all_joints
    ##################################################################################################
    
    #Loop through trajectory points:
    
    success = 0
    collisions = 0
    oob = 0
    time_taken = time()
    all_joints = []
    
    for k in range(len(cur_traj) - 1):
        
        dist_pos = 100 #??? This is getting reassigned later anyways
        
        marker_pos = marker_poses[k] # Get the kth point in the trajectory

        #Get the position and orientation of the target object, 
        #and find the distance between the target object and the kth marker
        targetPos, targetOrn = p.getBasePositionAndOrientation(body_ids[0]) 
        dist_pos = np.linalg.norm(marker_pos[0:2] - np.array(targetPos)[0:2])
        
        marker_orn = p.getQuaternionFromEuler([0, 0, marker_yaw]) #Convert marker yaw to Quaternion representation
        
        #Add a marker object to the environment, and get its object ID
        marker_obj, goal_suc = testcase1.add_marker_obj(marker_pos, marker_orn, half_extents=testcase1.current_target_size/2)
        
        #Append the ID of the marker object to the body IDs.
        body_ids.append(marker_obj)
        
        PUSH_ITER = 0
        
        obstacle_initial_poses = np.zeros((obstacle_num, 7))

        for j in range(obstacle_num):
            ob_start_pos, ob_start_orn = p.getBasePositionAndOrientation(body_ids[-1*(j+1)])
            obstacle_initial_poses[j, :3] = ob_start_pos[:]
            obstacle_initial_poses[j, 3:] = ob_start_orn[:]
        
        #Loop until the distance to current marker is less than threshold, and the number of pushes exceeds max no. of pushes
        while dist_pos > threshold_d and PUSH_ITER < MAX_PUSH_ITER:
            
            print(f"\rWaypoint: {k}\tPush: {PUSH_ITER}", end="")
            
            #Get the position and orientation of the target object again???
            targetPos, targetOrn = p.getBasePositionAndOrientation(body_ids[0])
            target_euler = p.getEulerFromQuaternion(targetOrn)  #Get the orientation in euler angles of the target
            
            #Get the yaw angle of the kth marker
            marker_yaw = marker_yaws[k]

            #GATHER THE CURRENT STATE (FOR MOVING TARGET OBJECT TO THE MARKER):
            # 1) Get the (x,y,theta) position and orientation of the target object: 
            cur_target_st = np.array([targetPos[0], targetPos[1], target_euler[2]], dtype=np.float64)
            # 2) Get the (x,y,theta) position and orientation of the marker object: 
            cur_target_goal = np.array([marker_pos[0], marker_pos[1], marker_yaw], dtype=np.float64)
            # 3) Stack these both to get the 6-vector state:
            cur_state = np.hstack((cur_target_st, cur_target_goal))
            
            #Convert state to a tensor and add an extra dimension on it, then store in dictionary:
            state = {
                'cur_state': torch.tensor(cur_state, dtype=torch.float, device=device).unsqueeze(0),
            }
            
            #Select the action with the highest value??? the output is a (1,1) action index
            action = select_action(state['cur_state'])
            
            #Check if the value inside the action tensor is in 0-15:
            if action.item() in range(0, 16):
                
                bottomPos, bottomOrn = [], []   #Initialize empty lists for bottom object's position and orientation
                
                #Get the target object's position and orientation again???
                targetPos, targetOrn = p.getBasePositionAndOrientation(body_ids[0])
                
                #Extract the size of the target object (l, b, h) from the test case
                current_target_obj_size = testcase1.current_target_size
                
                #Get the angle of direction of the push using the action chosen (Why are we doing modulus???)
                push_dir = push_directions[(action.item())%16]
                
                #Get the (x,y,z) starting coordinate and ending coordinate for the push by the manipulator.
                push_start, push_end = get_push_start_POSMAX(push_dir, bottomPos, bottomOrn,
                                                targetPos, targetOrn, current_target_obj_size, is_viz=False)
                
                
                succ, cur_joints = env.push(push_start, push_end)
                all_joints.extend(cur_joints)
                
                if not succ:
                    oob += 1

                new_target_pos, new_target_orn = p.getBasePositionAndOrientation(body_ids[0])
                target_euler = p.getEulerFromQuaternion(new_target_orn)

                new_target_st = np.array([new_target_pos[0], new_target_pos[1], target_euler[2]], dtype=np.float32)
                new_state = np.hstack((new_target_st, cur_target_goal))
                next_state = {
                    'cur_state': torch.tensor(new_state, dtype=torch.float, device=device).unsqueeze(0)
                }
                state=next_state
                reward = get_reward2(current_state=new_state, prev_state=state['cur_state'].squeeze().cpu().numpy())
                dist_pos = np.linalg.norm(cur_target_goal[0:2] - new_target_st[0:2])
                
            # ------------ Collision Cost -------------- #

            obstacle_final_poses = np.zeros((obstacle_num, 7))

            for j in range(obstacle_num):
                ob_end_pos, ob_end_orn = p.getBasePositionAndOrientation(body_ids[-1*(j+1)])
                obstacle_final_poses[j, :3] = ob_end_pos[:]
                obstacle_final_poses[j, 3:] = ob_end_orn[:]

            pose_diff = obstacle_final_poses - obstacle_initial_poses
            print(" Pose difference = ", np.linalg.norm(pose_diff))
            
            if np.linalg.norm(pose_diff) > 0.005:
                collisions += 1
            
            #collision_cost += 100000 * np.linalg.norm(pose_diff)

            # -------------------------------------------- #

            PUSH_ITER += 1
        
        if INVALID_POS==True:
            break
            
    # ------------ METRICS ---------------------- #
    
    error_threshold = 0.01
    targetPos, targetOrn = p.getBasePositionAndOrientation(body_ids[0])
    error = np.linalg.norm(targetPos[:2] - cur_traj[-1, :2])
    
    time_taken = time() - time_taken
    
    joint_cost = get_joint_cost(all_joints)
    
    st_time = time()
    
    # sample_costs.append(cost)
    # sample_joint_trajectories.append(np.array(all_joints))
    print(f"{ITER}. Cost: {joint_cost} -------------------------")
    # print(f"Time to get the joint cost: {time()-st_time}")
    return error, collisions, oob, time_taken, joint_cost

In [9]:
#CURRENT SCENE_ID FOR SIMULATION IS 5

env = Environment(gui=True)
env.reset()
threshold_d = 0.025

success, collisions, oob, time_taken, joint_cost = run_policy(env = env, 
                                                              cur_traj = best_traj[1:].copy(), 
                                                              START_PT = best_traj[0].copy(), 
                                                              threshold_d = threshold_d, 
                                                              WORKSPACE_LIMITS = WORKSPACE_LIMITS, 
                                                              MAX_PUSH_ITER=20,
                                                              obstacle_num = 2)

Obstacle Locations:  [[ 0.45 -0.05]
 [ 0.65 -0.1 ]
 [ 0.35  0.2 ]]
Waypoint: 0	Push: 0 Pose difference =  0.0009683087361251571
Waypoint: 0	Push: 1 Pose difference =  0.0019615055462643164
Waypoint: 0	Push: 2 Pose difference =  0.0027864509825677943
Waypoint: 0	Push: 3 Pose difference =  0.0037696684224284133
Waypoint: 0	Push: 4 Pose difference =  0.004585817283072956
Waypoint: 0	Push: 5 Pose difference =  0.0055505667455890435
Waypoint: 0	Push: 6 Pose difference =  0.006359082955125694
Waypoint: 0	Push: 7 Pose difference =  0.007310515939216328
Waypoint: 0	Push: 8 Pose difference =  0.008272130610851957
Waypoint: 0	Push: 9 Pose difference =  0.009195434691206598
Waypoint: 0	Push: 10 Pose difference =  0.010023174355019553
Waypoint: 0	Push: 11 Pose difference =  0.010918016509544787
Waypoint: 0	Push: 12 Pose difference =  0.011790893591803074
Waypoint: 0	Push: 13 Pose difference =  0.011864176280976033
Waypoint: 0	Push: 14 Pose difference =  0.011499347596927457
Waypoint: 0	Push: 15 Po

In [1]:
print("For Best Trajectory in " + save_folder, " : ")
print("Error between final point and goal point of trajectory = ", success)
print("Number of times object goes out of bounds = ", oob)
print("Time taken to complete trajectory = ", time_taken)
print("Total Joint Cost = ", joint_cost)

print(success, collisions, oob, time_taken, joint_cost)

NameError: name 'save_folder' is not defined